## Preprocessing

In [27]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import keras_tuner as kt

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [28]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#initally dropped the NAME column, but decided to include it.
#including the name column increased the accuracy of the model.
application_df = application_df.drop(columns = ['EIN'])#,'NAME')
application_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [29]:
# Determine the number of unique values in each column.
application_df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [30]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [31]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
app_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = app_counts[app_counts < 500].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [32]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [33]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_count = application_df['CLASSIFICATION'].value_counts() 
class_count[class_count > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [34]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = class_count[class_count < 1000].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [35]:
name_count = application_df['NAME'].value_counts()

In [43]:
# name column binning
# use the variable name `name_to_replace`
name_to_replace = name_count[name_count < 100].index.tolist()

# Replace in dataframe
for name in name_to_replace:
    application_df['NAME'] = application_df['NAME'].replace(name,"Other")
    
# Check to make sure binning was successful
application_df['NAME'].value_counts()

Other                                                                 25987
PARENT BOOSTER USA INC                                                 1260
TOPS CLUB INC                                                           765
UNITED STATES BOWLING CONGRESS INC                                      700
WASHINGTON STATE UNIVERSITY                                             492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                         408
PTA TEXAS CONGRESS                                                      368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC                           331
ALPHA PHI SIGMA                                                         313
TOASTMASTERS INTERNATIONAL                                              293
MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS                       287
LITTLE LEAGUE BASEBALL INC                                              277
INTERNATIONAL ASSOCIATION OF LIONS CLUBS                                266
MOMS CLUB   

In [44]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummy = pd.get_dummies(application_df)

In [45]:
# Split our preprocessed data into our features and target arrays
y = application_dummy['IS_SUCCESSFUL']
X = application_dummy.drop(columns = ['IS_SUCCESSFUL']).values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [46]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [47]:
# Create a method that creates a new Sequential model with hyperparameter options
#keras tuner to determine which parameters will give us the best accuracy
def create_model(hp): 
    nn_model = tf.keras.models.Sequential()
    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=100,
        step=2), activation=activation, input_dim=74))
    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=40,
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [48]:
# Create a `Hyperband()` tuner instance
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=25,
    hyperband_iterations=2,
    overwrite=True)

In [49]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=25,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 00m 38s]
val_accuracy: 0.7554519176483154

Best val_accuracy So Far: 0.756618082523346
Total elapsed time: 00h 14m 42s
INFO:tensorflow:Oracle triggered exit


In [50]:
best_params = tuner.get_best_hyperparameters()[0]
best_params.values

{'activation': 'relu',
 'first_units': 47,
 'num_layers': 3,
 'units_0': 17,
 'units_1': 27,
 'units_2': 37,
 'units_3': 27,
 'units_4': 7,
 'tuner/epochs': 9,
 'tuner/initial_epoch': 3,
 'tuner/bracket': 2,
 'tuner/round': 1,
 'tuner/trial_id': '0031'}

In [51]:
best_model = tuner.get_best_models(1)[0]

In [52]:
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5030 - accuracy: 0.7566 - 332ms/epoch - 1ms/step
Loss: 0.5029551982879639, Accuracy: 0.756618082523346


In [53]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 47)                3525      
                                                                 
 dense_1 (Dense)             (None, 17)                816       
                                                                 
 dense_2 (Dense)             (None, 27)                486       
                                                                 
 dense_3 (Dense)             (None, 37)                1036      
                                                                 
 dense_4 (Dense)             (None, 1)                 38        
                                                                 
Total params: 5,901
Trainable params: 5,901
Non-trainable params: 0
_________________________________________________________________


In [54]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=47, input_dim=74, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=17, activation="relu"))
nn.add(tf.keras.layers.Dense(units=27, activation="relu"))
nn.add(tf.keras.layers.Dense(units=37, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 47)                3525      
                                                                 
 dense_6 (Dense)             (None, 17)                816       
                                                                 
 dense_7 (Dense)             (None, 27)                486       
                                                                 
 dense_8 (Dense)             (None, 37)                1036      
                                                                 
 dense_9 (Dense)             (None, 1)                 38        
                                                                 
Total params: 5,901
Trainable params: 5,901
Non-trainable params: 0
_________________________________________________________________


In [55]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [56]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5169 - accuracy: 0.7442
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4933 - accuracy: 0.7569
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4908 - accuracy: 0.7588
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4884 - accuracy: 0.7589
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4869 - accuracy: 0.7608
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4862 - accuracy: 0.7607
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4849 - accuracy: 0.7617
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4839 - accuracy: 0.7619
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4844 - accuracy: 0.7618
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4834 - accura

804/804 [==============================] - 1s 1ms/step - loss: 0.4726 - accuracy: 0.7696
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4723 - accuracy: 0.7682
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4725 - accuracy: 0.7693
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4721 - accuracy: 0.7682
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4729 - accuracy: 0.7683
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4723 - accuracy: 0.7682
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4720 - accuracy: 0.7685
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4724 - accuracy: 0.7685
Epoch 89/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4722 - accuracy: 0.7670
Epoch 90/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4722 - accuracy: 

In [58]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5045 - accuracy: 0.7503 - 294ms/epoch - 1ms/step
Loss: 0.5045482516288757, Accuracy: 0.7503206729888916


In [59]:
# Evaluate the model using the test data
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5030 - accuracy: 0.7566 - 202ms/epoch - 755us/step
Loss: 0.5029551982879639, Accuracy: 0.756618082523346


In [61]:
# Export our model to HDF5 file
nn.save('soup_model.h5')
best_model.save('soup_model_2.h5')